In [1]:
import math
import numpy as np

In [2]:
NodesCondition = []
ForcesCondition = []
def UBoundaryCondition(nU,u,i):
    nU[i][0] = u
    NodesCondition.append(i)

def FBoundaryCondition(nF,f,i):
    nF[i][0] = f
    ForcesCondition.append(i)
    
def AssemblyStiffness(nStiffnessMatrix,k,i,j):
    nStiffnessMatrix[i][i] += k
    nStiffnessMatrix[i][j] += -k
    nStiffnessMatrix[j][i] += -k
    nStiffnessMatrix[j][j] += k
def Initialize(nStiffnessMatrix,nU,nF):
    for i in range(0,Nodes):
        nU[i][0] = math.pi
        nF[i][0] = math.pi
    for i in range(0,NumberOfElement):
        AssemblyStiffness(nStiffnessMatrix,K[i][0],int(K[i][1]),int(K[i][2]))

def PreSolvingStiffness(nStiffnessMatrix):
    nsize = Nodes-len(NodesCondition)
    newStiffness = np.zeros((nsize,nsize))
    contr = -1
    for i in range(0,Nodes):
        contc = -1
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        for j in range(0,Nodes):
            flagc = False
            for k in range(0,len(NodesCondition)):
                if(j == NodesCondition[k]):
                    flagc = True
                    break
            if(flagc):
                continue
            contc += 1
            newStiffness[contr][contc] = nStiffnessMatrix[i][j]
    return newStiffness

def PreSolvingF(nF,nS,nU):
    nsize = Nodes-len(NodesCondition)
    newF = np.zeros(nsize).reshape(nsize,1)
    contr = -1
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            for k in range(0,Nodes):
                nF[k][0] = nF[k][0]-nS[k][i]*nU[i][0]
            continue

    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        newF[contr][0] = nF[i][0]
    
    return newF
                      

def Solve(nStiffnessMatrix,nU,nF):
    newStiffness = PreSolvingStiffness(nStiffnessMatrix)
    newF = PreSolvingF(nF,nStiffnessMatrix,nU)
    u = np.linalg.solve(newStiffness,newF)
    contr = -1
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        nU[i][0] = u[contr][0]
    nnF = np.matmul(StiffnessMatrix,nU)
    return nU,nnF



In [3]:
NodesCondition = []
Nodes = 3

E = 210*1e6 #GPA
A = 0.003 #m2
L = 2.5 #m

K = np.array([[E*A/1.5,0,1],[E*A/1,1,2]])

NumberOfElement = len(K)

StiffnessMatrix = np.zeros((Nodes,Nodes))
U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)
UBoundaryCondition(U,0,0)
UBoundaryCondition(U,0.002,2)
FBoundaryCondition(F,-10,1)

U,F=Solve(StiffnessMatrix,U,F)

print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)

Stiffness Matrix:
 [[ 420000. -420000.       0.]
 [-420000. 1050000. -630000.]
 [      0. -630000.  630000.]] 

Displacements:
 [[0.        ]
 [0.00119048]
 [0.002     ]] 

Forces:
 [[-500.]
 [ -10.]
 [ 510.]]


In [4]:
NodesCondition = []
Nodes = 6

E = 210*1e6 #GPA
#A = 0.003 #m2
L = 3 #m
b0 = 0.002
bf = 0.012
K = []
#K = np.array([[E*A/1.5,0,1],[E*A/1,1,2]])
A = []
for i in range(0,Nodes):
    b = b0+i*((bf-b0)/(Nodes-1))
    A.append(b0+(bf-b0)*i*(1/(Nodes-1))+0.001)
for i in range(0,Nodes-1):
    K.append([E*A[i]/(L/(Nodes-1)),i,i+1])
    
NumberOfElement = len(K)

StiffnessMatrix = np.zeros((Nodes,Nodes))
U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)
UBoundaryCondition(U,0,Nodes-1)
#UBoundaryCondition(U,0.002,2)
FBoundaryCondition(F,-18,0)
for i in range(1,Nodes-1):
    FBoundaryCondition(F,0,i)

U,F=Solve(StiffnessMatrix,U,F)

print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)

Stiffness Matrix:
 [[ 1050000. -1050000.        0.        0.        0.        0.]
 [-1050000.  2800000. -1750000.        0.        0.        0.]
 [       0. -1750000.  4200000. -2450000.        0.        0.]
 [       0.        0. -2450000.  5600000. -3150000.        0.]
 [       0.        0.        0. -3150000.  7000000. -3850000.]
 [       0.        0.        0.        0. -3850000.  3850000.]] 

Displacements:
 [[-4.51651206e-05]
 [-2.80222635e-05]
 [-1.77365492e-05]
 [-1.03896104e-05]
 [-4.67532468e-06]
 [ 0.00000000e+00]] 

Forces:
 [[-1.80000000e+01]
 [ 1.06581410e-14]
 [ 0.00000000e+00]
 [-1.77635684e-15]
 [ 0.00000000e+00]
 [ 1.80000000e+01]]
